## Imports

In [1]:
!pip install birt-gd
!pip install tslearn


[notice] A new release of pip available: 22.3 -> 23.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# utils
import os
import re

# basics
import numpy as np
import pandas as pd

# models
from birt import Beta4
from sklearn.cluster import (
    DBSCAN,
    OPTICS,
    KMeans,
    MeanShift,
    SpectralClustering,
)

# metrics
from sklearn.metrics import (
    adjusted_rand_score,
    calinski_harabasz_score,
    davies_bouldin_score,
    mutual_info_score,
    silhouette_score,
    v_measure_score,
)
from tqdm import tqdm
from tslearn.clustering import KernelKMeans

from utils.processing.transform import TransformPairwise, TransformPairwise2

np.random.seed(0)

2023-04-07 12:58:09.754230: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-07 12:58:10.235452: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-04-07 12:58:11.557533: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-04-07 12:58:11.557729: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

new method: 0:00:11.212716
           c0        c1        c2        c3        c4
0    0.538462  0.563545  0.542642  0.536789  0.537625
1    0.554348  0.562709  0.556020  0.532609  0.540134
2    0.581940  0.582776  0.547659  0.569398  0.581104
3    0.548495  0.523411  0.549331  0.571070  0.583612
4    0.571906  0.552676  0.548495  0.545151  0.554348
..        ...       ...       ...       ...       ...
295  0.570234  0.542642  0.546823  0.571070  0.558528
296  0.549331  0.559365  0.559365  0.555184  0.539298
297  0.522575  0.540134  0.548495  0.551003  0.576923
298  0.574415  0.585284  0.559365  0.568562  0.555184
299  0.551839  0.548495  0.530936  0.582776  0.568562

[300 rows x 5 columns]


## Generate special clusters

In [3]:
NEW = False

In [10]:
ROOT = os.getcwd()

DIR = "data"
NAMEs = [
    "aniso",
    "blobs",
    "varied",
    "noisy_moons",
    "noisy_circles",
    "no_structure",
]

In [11]:
URLs = [os.path.join(ROOT, DIR, i) for i in NAMEs]

In [12]:
data_all = {
    i.split("/")[-1]: pd.read_csv(os.path.join(i, i.split("/")[-1] + ".csv"))
    for i in URLs
}

## Running for all clusterings dataset

In [13]:
_X, _Y = {}, {}
for i in data_all.keys():
    _X[i] = data_all[i].drop("labels", axis=1).values
    _Y[i] = data_all[i]["labels"].values

In [14]:
dir_result = "results"
models = {
    "kmeans": KMeans,
    "dbscan": DBSCAN,
    "kernel_kmeans": KernelKMeans,
    "spectral_clustering": SpectralClustering,
    "mean_shift": MeanShift,
    "optics": OPTICS,
}
params = {
    "kmeans": [{"n_clusters": i} for i in range(1, 9, 1)],
    "dbscan": [
        {"eps": round(i, 1), "min_samples": 2}
        for i in np.arange(0.1, 1.0, 0.1)
    ],
    "spectral_clustering": [
        {
            "n_clusters": i,
            "eigen_solver": "arpack",
            "affinity": "nearest_neighbors",
        }
        for i in range(1, 9, 1)
    ],
    "mean_shift": [{}],
    "kernel_kmeans": [
        {"n_clusters": i, "kernel": "gak", "random_state": 0}
        for i in np.arange(1, 9, 1)
    ],
    "optics": [{"min_samples": 2}],
}

In [20]:
models_name = []
for name, model in params.items():
    for name_to_process in model:
        text = str(list(name_to_process.items()))
        text = re.sub(",+", "_", text)
        text = text.replace(".", "_")
        text = re.sub("[^0-9a-zA-Z_-]", "", text)
        text = str(name + "_" + text).replace(" ", "_")
        models_name.append(text)

In [21]:
combination_models = TransformPairwise2().combination_models(models, params)

for _name in tqdm(_X.keys()):
    print(_name)
    X_dataset_i = _X[_name]
    Y_dataset_i = _Y[_name]

    for model in combination_models:
        model.fit(X_dataset_i)

    results = {}
    for idxx, i in enumerate(models_name):
        if hasattr(combination_models[idxx], "labels_"):
            results[i] = combination_models[idxx].labels_

    data_results = pd.DataFrame(results)

    tp = TransformPairwise2(data_results, 4)
    pij = tp.generate_pij_matrix(data_results)
    pij["average_model"] = pij.mean(axis=1)
    pij["optimal_clustering"] = pij.max(axis=1)

    b4 = Beta4(
        learning_rate=0.1,
        epochs=10000,
        n_respondents=pij.shape[1],
        n_items=pij.shape[0],
        n_inits=1000,
        # n_workers=-1,
        random_seed=1,
        tol=10 ** (-8),
        set_priors=True,
    )
    b4.fit(pij.values)

    abilities_orderned = pd.DataFrame(
        b4.abilities, index=pij.columns
    ).sort_values(0, ascending=False)
    abilities_orderned.columns = ["abilities"]

    metrics = [
        ("v_measure", v_measure_score, True),
        ("mutual_info", mutual_info_score, True),
        ("adjusted_rand_score", adjusted_rand_score, True),
        ("calinski_harabasz", calinski_harabasz_score, False),
        ("davies_bouldin", davies_bouldin_score, False),
        ("silhouette", silhouette_score, False),
    ]

    data_metrics = pd.DataFrame(
        columns=["abilities"] + [i[0] for i in metrics],
        index=abilities_orderned.index,
    )
    data_metrics["abilities"] = abilities_orderned.abilities
    data_metrics = data_metrics.T
    params_metric = {
        "mood1": {"labels_true": Y_dataset_i},
        "mood2": {"X": X_dataset_i},
    }

    model_metrics = []
    for model in data_metrics.columns:
        for metric in metrics:
            if model not in ["optimal_clustering", "average_model"]:
                if metric[-1]:
                    _params = params_metric["mood1"] | {
                        "labels_pred": results[model]
                    }
                else:
                    _params = params_metric["mood2"] | {
                        "labels": results[model]
                    }
                try:
                    data_metrics.loc[metric[0], model] = metric[1](**_params)
                except:
                    data_metrics.loc[metric[0], model] = np.nan
            else:
                data_metrics.loc[metric[0], model] = np.nan
    data_metrics = data_metrics.T

    dir_contents = [
        (
            "metrics",
            (
                "metrics.csv",
                data_metrics.sort_values("abilities", ascending=False),
            ),
            (None),
        ),
        (
            "pij",
            ("pij_true.csv", pij),
            ("pij_pred.csv", pd.DataFrame(b4.pij, columns=pij.columns)),
        ),
        (
            "params",
            (
                "abilities.csv",
                pd.DataFrame(
                    b4.abilities, index=pij.columns, columns=["abilities"]
                ),
            ),
            (
                "diff_disc.csv",
                pd.DataFrame(
                    {
                        "difficulty": b4.difficulties,
                        "discrimination": b4.discriminations,
                    }
                ),
            ),
        ),
        ("labels", ("labels.csv", pd.DataFrame(results)), (None)),
    ]

    dir_save_dataset_i = os.path.join(ROOT, dir_result, _name)
    url_save_dataset_i = [
        os.path.join(dir_save_dataset_i, i[0]) for i in dir_contents
    ]
    for url in url_save_dataset_i:
        if not os.path.exists(url):
            os.makedirs(url)

    for content, url in zip(dir_contents, url_save_dataset_i):
        for data_save in content[1:]:
            if data_save is not None:
                print(os.path.join(url, data_save[0]))
                data_save[1].to_csv(os.path.join(url, data_save[0]))

  0%|                                                                                                                                                                                        | 0/6 [00:00<?, ?it/s]

aniso


/home/manuel/.local/lib/python3.10/site-packages/tslearn/utils/utils.py:88: UserWarning: 2-Dimensional data passed. Assuming these are 500 1-dimensional timeseries
  warnings.warn('2-Dimensional data passed. Assuming these are '
/home/manuel/.local/lib/python3.10/site-packages/tslearn/utils/utils.py:88: UserWarning: 2-Dimensional data passed. Assuming these are 500 1-dimensional timeseries
  warnings.warn('2-Dimensional data passed. Assuming these are '
/home/manuel/.local/lib/python3.10/site-packages/tslearn/utils/utils.py:88: UserWarning: 2-Dimensional data passed. Assuming these are 500 1-dimensional timeseries
  warnings.warn('2-Dimensional data passed. Assuming these are '
/home/manuel/.local/lib/python3.10/site-packages/tslearn/utils/utils.py:88: UserWarning: 2-Dimensional data passed. Assuming these are 500 1-dimensional timeseries
  warnings.warn('2-Dimensional data passed. Assuming these are '
/home/manuel/.local/lib/python3.10/site-packages/tslearn/utils/utils.py:88: UserWarn

  5%|████████                                                                                                                                                                  | 471/10000 [00:07<02:00, 79.17it/s]


  8%|█████████████                                                                                                                                                             | 769/10000 [00:11<02:26, 62.84it/s]


...

 17%|████████████████████████████▏                                                                                                                                            | 1668/10000 [00:29<02:45, 50.35it/s]


 19%|███████████████████████████████▉                                                                                                                                         | 1887/10000 [00:33<02:58, 45.46it/s]

Model converged at the 1889th epoch


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [1:06:00<00:00, 660.13s/it]

/home/manuel/Documents/manuel/tcc/tcc01/results/no_structure/metrics/metrics.csv
/home/manuel/Documents/manuel/tcc/tcc01/results/no_structure/pij/pij_true.csv
/home/manuel/Documents/manuel/tcc/tcc01/results/no_structure/pij/pij_pred.csv
/home/manuel/Documents/manuel/tcc/tcc01/results/no_structure/params/abilities.csv
/home/manuel/Documents/manuel/tcc/tcc01/results/no_structure/params/diff_disc.csv
/home/manuel/Documents/manuel/tcc/tcc01/results/no_structure/labels/labels.csv
